In [ ]:
from io import BytesIO
import random

import util
from test_framework.address import program_to_witness
from test_framework.key import ECKey, ECPubKey, generate_schnorr_nonce
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness, sha256
from test_framework.musig import generate_musig_key, aggregate_schnorr_nonces, sign_musig, aggregate_musig_signatures
from test_framework.script import CScript, OP_1, TaprootSignatureHash, SIGHASH_ALL_TAPROOT

# 2.1 Taproot Outputs

* Part 1: Generating a segwit v1 output address
* Part 2: Sending funds from the Bitcoin Core wallet
* Part 3: Constructing a transaction to spend the segwit v1 output
    * Spending a segwit v1 output with a MuSig public key

In this chapter, we introduce segwit v1 outputs, which are defined in [bip-taproot](https://github.com/bitcoinops/bips/blob/v0.1/bip-taproot.mediawiki). Segwit v1 outputs can be spent in two ways:

* **Key path** spending, which treats the witness program as a public key, and permits spending using a signature from that public key.
* **Script path** spending, which allows a pre-committed script to be used to spend the output. Script path spending will be fully described in chapters 2.2, 2.3 and 2.4.

By using the MuSig pubkey and signature aggregation protocol described in chapter 1.2, key path spending can be used to encumber an output to an n-of-n multisig policy in a way that is indistinguishable from a single-key output and spend.

The first half of this chapter shows an example of sending funds to a segwit v1 address using the Bitcoin Core wallet, and then manually constructing a transaction that spends that output using the new bip-taproot key path spending rules.

![test](images/segwit_version1_1.jpg)

There is then a coding exercise to send funds to a MuSig aggregate pubkey address, and then construct a transaction that spends from that address using the bip-taproot key path spending rules, which follows the same 

## Part 1. Generating a segwit v1 output

Segwit v1 follows the same output script pattern as segwit v0:

* Segwit output: **`[1B Version]` `[segwit program]`**
* Segwit v0 output: **`[00]` `[20-Byte public key digest]`** (P2WPKH) or **`[00]` `[32-Byte script digest]`** (P2WSH)
* Segwit v1 output: **`[01]` `[33-Byte public key]`**

The 33-Byte public key encoding is similar to that of legacy compressed pubkeys, but with a different oddness byte.

* Y-coordinate: even - **`[00]`** or odd - **`[01]`**
* X-coordinate: **`[32B x-coordinate]`**
* 33 byte pubkey encoding: **`[1B oddness] [32B x-coordinate]`**

### Spending a segwit v1 output with the key path

Unlike segwit v0 outputs, v1 outputs look the same for script or key paths(unlike v0 which separates into P2WPKH and P2WSH). In this chapter we will focus on spending the key path.

![test](images/segwit_version1_0.jpg)

The output can be spent along the **key path** by providing a valid signature for the pubkey in the output's scriptPubKey. The spending witness is simply **`[sig]`**.

The output can be spent along the **script path** if public key was tweaked with a valid taproot. See chapters 2.2 and 2.3 for further details.

#### 2.1.1 Example: Constructing a segwit v1 output

In this example, we construct segwit v1 output for spending along the key path. We generate a key pair, encode the public key using the bip-schnorr and bip-taproot pubkey encoding rules, and then encode the witness version and witness program to a bech32 address.

In [ ]:
# Key pair generation
privkey = ECKey().generate()
pubkey = privkey.get_pubkey()
print("Pubkey is {}\n".format(pubkey.get_bytes().hex()))

# Create witness program ([1B oddness] [32B x-coordinate])
pubkey_data = pubkey.get_bytes()
program = bytes([pubkey_data[0] & 1]) + pubkey_data[1:]
print("Witness program is {}\n".format(program.hex()))

# Create (regtest) bech32 address
version = 0x01
address = program_to_witness(version, program)
print("bech32 address is {}".format(address))

## Part 2. Sending funds from the Bitcoin Core wallet

Next, we send funds to the segwit v1 address that we just generated. We'll create send the funds from a Bitcoin Core wallet, which is able to send outputs to segwit v1 addresses.

#### Example 2.1.2: Startup TestWrapper to initialize a regtest node and wallet
Only run setup once, or after a clean shutdown.

In [ ]:
test = util.TestWrapper()
test.setup()

version = test.nodes[0].getnetworkinfo()['subversion']
print("\nClient version is {}".format(version))

#### Example 2.1.3: Generate coins for the wallet

In [ ]:
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print('Balance: {}'.format(balance))

assert balance > 1

#### Example 2.1.4: Send funds from the Bitcoin Core wallet to the segwit address

In [ ]:
# Send wallet transaction to segwit address
amount_btc = 0.05
txid = test.nodes[0].sendtoaddress(address, amount_btc)

# Decode wallet transaction
tx_hex = test.nodes[0].getrawtransaction(txid) 
decoded_tx = test.nodes[0].decoderawtransaction(tx_hex)

print("Transaction:\n{}\n".format(decoded_tx))

# Reconstruct wallet transaction locally
tx = CTransaction()
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# We can check if the transaction was correctly deserialized
assert txid == decoded_tx["txid"]

# The wallet randomizes the change output index for privacy
# Loop through the outputs and return the first where the scriptPubKey matches the segwit v1 output
output_index, output = next(out for out in enumerate(tx.vout) if out[1].scriptPubKey == CScript([OP_1, program]))

print("Segwit v1 output is {}".format(output))
print("Segwit v1 output value is {}".format(output.nValue))
print("Segwit v1 output index is {}".format(output_index))

## Part 3. Constructing a transaction to spend the segwit v1 output

We are now going to manually contruct, sign and broadcast a transaction which spends the segwit v1 output.

To do that we create a `CTransaction` object and populate the data members:

 * `nVersion`
 * `nLocktime`  
 * `tx_vin` (list of `CTxIn` objects)
 * `tx_vout` (list of `CTxOut` objects)
 * `tx.wit.vtxinwit` (list of `CTxInWitness` objects)

#### Example 2.1.5: Construct `CTransaction` and populate inputs

In [ ]:
# Construct transaction
spending_tx = CTransaction()

# Populate the transaction version
spending_tx.nVersion = 1

# Populate the locktime
spending_tx.nLockTime = 0

# Populate the transaction inputs
outpoint = COutPoint(tx.sha256, output_index)
spending_tx_in = CTxIn(outpoint)
spending_tx.vin = [spending_tx_in]

print("Spending transaction:\n{}".format(spending_tx))

#### Example 2.1.6: Populate outputs

We'll generate an output address in the Bitcoin Core wallet to send the funds to, determine the fee, and then populate the spending_tx with an output to that address.

In [ ]:
# Generate new Bitcoin Core wallet address
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet
amount_sat = int(amount_btc * 100000000)
dest_output = CTxOut(nValue=amount_sat - min_fee, scriptPubKey=scriptpubkey)
spending_tx.vout = [dest_output]

print("Spending transaction:\n{}".format(spending_tx))

#### Example 2.1.7: Sign the transaction with a schnorr signature

bip-taproot defines the following sighash flags:
* Legacy sighash flags:
  * `0x01` - **SIGHASH_ALL**
  * `0x02` - **SIGHASH_NONE**
  * `0x03` - **SIGHASH_SINGLE**
  * `0x81` - **SIGHASH_ALL | SIGHASH_ANYONECANPAY**
  * `0x82` - **SIGHASH_NONE | SIGHASH_ANYONECANPAY**
  * `0x83` - **SIGHASH_SINGLE | SIGHASH_ANYONECANPAY**
* New sighash flag:
  * `0x00` - same semantics `0x01` **SIGHASH_ALL**

Append the sighash flag to the signature `[R_x, s]` with the sighash byte if not `0x00`.

In [ ]:
# Generate the taproot signature hash for signing
# SIGHASH_ALL_TAPROOT is 0x00
sighash = TaprootSignatureHash(spending_tx, [output], SIGHASH_ALL_TAPROOT, input_index=0)
 
# All schnorr sighashes except SIGHASH_ALL_TAPROOT require
# the hash_type appended to the end of signature
sig = privkey.sign_schnorr(sighash)

print("Signature: {}".format(sig.hex()))

#### Example 2.1.8: Add the witness and test acceptance of the transaction

In [ ]:
# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(sig)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness

# vtxinwit is a list of the witness data (signatures, etc.)
spending_tx.wit.vtxinwit.append(witness_in)

print("Spending transaction:\n{}\n".format(spending_tx))
 
# Serialize signed transaction for broadcast
spending_tx_str = spending_tx.serialize().hex()
 
# Test mempool acceptance
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']
print("Success!")

**Shutdown the TestWrapper (and all bitcoind instances)**

In [ ]:
test.shutdown()

### Spending a segwit v1 output with a MuSig public key

In this exercise, we'll use the Bitcoin Core wallet to create a transaction that sends to a 2-of-2 musig aggregate pubkey (segwit v1 address).

We'll then manually create a transaction that spends the MuSig output and sends an output back to the Bitcoin Core wallet. We'll use Bitcoin Core's `testmempoolaccept()` RPC method to verify that the transaction is valid.

#### Start the TestWrapper (and bitcoind node)

In [ ]:
test = util.TestWrapper()
test.setup()

version = test.nodes[0].getnetworkinfo()['subversion']
print("\nClient version is {}".format(version))

#### 2.1.9 _Programming Exercise:_ Generate segwit v1 addresses for a 2-of-2 MuSig aggregate pubkey

In this exercise, we create a 2-of-2 aggregate MuSig public key

In [ ]:
# Generate individual key pairs
privkey1 = ECKey().generate()
privkey2 = ECKey().generate()
pubkey1 = privkey1.get_pubkey()
pubkey2 = privkey2.get_pubkey()

# Generate a 2-of-2 aggregate MuSig key
# Method: generate_musig_key(ECPubKey_list)
c_map, agg_pubkey =  # TODO: implement

# Multiply individual keys with challenges
privkey1_c =  # TODO: implement
privkey2_c =  # TODO: implement
pubkey1_c =  # TODO: implement
pubkey2_c =  # TODO: implement

# Create a segwit v1 address for the MuSig aggregate pubkey
# Method: address = program_to_witness(version_int, program_bytes)
program_musig =  # TODO: implement
address_musig =  # TODO: implement
print("2-of-2 musig: ", address_musig)

#### Example 2.1.10: Create a transaction in the Bitcoin Core wallet sending outputs to the segwit v1 addresses

In [ ]:
# Generate coins in Bitcoin Core wallet
blocks = test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print('Balance:', balance)

# Create Bitcoin Core wallet transaction sending to the MuSig segwit v1 address
amount_btc = 0.05
txid = test.nodes[0].sendmany(amounts={address_musig: amount_btc})

# Reconstruct wallet transactions locally
tx_hex = test.nodes[0].getrawtransaction(txid)

tx_musig = CTransaction()
tx_musig.deserialize(BytesIO(bytes.fromhex(tx_hex)))

# Rehash recalculates the txid (needed later)
tx_musig.rehash()

print("Transaction:\n{}\n".format(tx_musig))

# Loop through outputs and find the MuSig output
output_index, output = next(out for out in enumerate(tx_musig.vout) if out[1].scriptPubKey == CScript([OP_1, program_musig]))

print("MuSig output is {}".format(output))
print("MuSig output value is {}".format(output.nValue))
print("MuSig output index is {}".format(output_index))

#### 2.1.11 _Programming Exercise:_ Instantiate a CTransaction object and populate the version, locktime and inputs

In [ ]:
# Construct transaction which spends the musig segwit v1 output
spending_tx = CTransaction()
spending_tx.nVersion =  # TODO: implement
spending_tx.nLockTime =  # TODO: implement
spending_tx.vin =  # TODO: implement

# Generate new Bitcoin Core wallet address
# Method: addr_string = test.nodes[0].getnewaddress(address_type="bech32")
# Method: decode addr_string with test.nodes[0].getaddressinfo(addr_string)
dest_addr =  # TODO: implement
scriptpubkey =  # TODO: implement
print("Destination address: {}\n".format(dest_addr))

# Determine minimum fee required for mempool acceptance
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet
# Tip: Construct output with CTxOut(nValue=value_int, scriptPubKey=script_bytes)
# Tip: CTransaction.vout is a list of CTxOut objects.
spending_tx.vout =  # TODO: implement
print("Spending transaction:\n{}\n".format(spending_tx))

#### 2.1.12 _Programming Exercise:_ Create a valid bip-schnorr signature for the MuSig aggregate pubkey

In this exercise, we create a signature for the aggregate pubkey, add it to the witness, and then test that the transaction is accepted by the mempool.

In [ ]:
# Create sighash for ALL.
# Tip: TaprootSignatureHash(tx, [output_list], hash_type = int, input_index = int, scriptpath = bool)
sighash_musig =  # TODO: implement

# Generate individual nonces for participants and an aggregate nonce point
# Remember to negate the individual nonces if necessary
# Method: generate_schnorr_nonce()
# Method: aggregate_schnorr_nonces(nonce_list)
nonce1 = # TODO: implement
nonce2 = # TODO: implement
R_agg, negated =  # TODO: implement

# Create an aggregate signature
# Method: sign_musig(privkey, nonce, R_agg, agg_pubkey, sighash_musig)
# Method: aggregate_musig_signatures(partial_signature_list, R_agg)
s1 = # TODO: implement
s2 = # TODO: implement
sig_agg =  # TODO:implement
print("Aggregate signature is {}\n".format(sig_agg.hex()))

# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(  # TODO: implement
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

# Serialize Schnorr transaction for broadcast
spending_tx_str = spending_tx.serialize().hex()

# Test mempool acceptance
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']
print("Success!")

In [ ]:
test.shutdown()

**Congratulations!** In this chapter, you have:

- Learned how to create a segwit v1 output and derive its bech32 address.
- Started a Bitcoin Core full node (in regtest mode), generated 101 blocks and sent a transaction output to the segwit v1 output.
- Constructed a transaction that spends the segwit v1 output back to the wallet using the key path.
- Repeated the above steps for a MuSig aggregate pubkey and signature.